In [1]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw
from bw2data import parameters
from sympy.physics.units import years
from sympy.stats.crv_types import LogNormalDistribution
from sympy import init_printing
import lca_algebraic as agb
from dotenv import load_dotenv

# Custom utils defined for the parameterization
from lca_algebraic import *
from lca_algebraic.stats import *

# Pretty print for Sympy
init_printing()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, lognorm, expon, beta, uniform, triang, kstest

In [3]:
BW_PROJECT = 'lca-alg-2' # insert your project name here
bd.projects.set_current(BW_PROJECT)

EI_DB = 'ecoinvent-3.9.1-cutoff' # name of ecoinvent database in your project
# We use a separate DB for defining our foreground model / activities
USER_DB = "MyForeground"

In [4]:
# This is better to cleanup the whole foreground model each time, and redefine it in the notebook (or a python file)
# instead of relying on a state or previous run.
# Any persistent state is prone to errors.
agb.resetDb(USER_DB)

# Parameters are stored at project level : 
# Reset them also
# You may remove this line if you import a project and parameters from an external source (see loadParam(..))
agb.resetParams()

# Overview of the databases
agb.list_databases()

[WARNING] Db MyForeground was here. Reseting it


,backend,nb_activities,type
name,,,
biosphere3,sqlite,4718,biosphere
ecoinvent-3.9.1-cutoff,sqlite,21238,background
MyForeground,sqlite,0,foreground


# Defining input parameters for the parametrization

The following parameters are included:

- ore grade
- mining technique mix (open cast, underground, ISL)
- mining depth
- tailings emissions
- mining energy mix 

lca_algebraic supports seven distribution types: 
- Fixed 
- Uniform 
- Triangle
- Normal
- Log-normal
- Beta
- Statistic weight

In [19]:
# Float parameters 
og_cu_world = newFloatParam(
    'ore_grade_copper', 
    default=cu_mean,
    std= cu_std,
    min=cu_min,
    max=cu_max,
    unit='dimensionless',
    distrib=DistributionType.NORMAL,
    description="From statistical analysis of ore grade data from Mudd et al (2018)",
    label="Copper ore grade",
    dbname=USER_DB, 
    save=False #By default, new parameters are kept in memory but also persisted in the project (unless save=False)
)

og_ni_world = newFloatParam(
    'ore_grade_nickel', 
    default=ni_mean,
    std= ni_std,
    min=ni_min,
    max=ni_max,
    unit='dimensionless',
    distrib=DistributionType.NORMAL,
    description="From statistical analysis of ore grade data from Mudd et al (2014)",
    label="Nickel ore grade",
    dbname=USER_DB, 
    save=False #By default, new parameters are kept in memory but also persisted in the project (unless save=False)
)

In [ ]:
mining_tech_mix = {'open_pit':.161/.935,
                'underground':.200/.935,
                'ISL':0.574/.935}

In [ ]:
mining_energy_mix = {'diesel':0.3,
                        'electricity':0.7}

In [ ]:
mining_electricity_switch = newEnumParam(
    'mining_electricity_switch', 
    label='Mining electricity, grid or diesel',
    values=['dieselgenerator',
            'grid'], # You can provide a statistical weight for each value, by using a dict
    default='dieselgenerator', 
    description="Choice of electricity source for mining",
    dbname=USER_DB)

In [ ]:
# Emissions of radon-222 from tailings
tailings_Rn222 = newFloatParam(
    'tailings_Rn222', 
    default=0.01951, min=0.01, max=1,
    distrib=DistributionType.TRIANGLE, # Distribution type, linear by default
    description="Rn222 from tailings, in Bq/s",
    label="Rn222 from tailings, in Bq/s",
    unit='Bq/s',
    dbname=USER_DB)

In [ ]:
# Conversion phase, heat consumption
conversion_heat = newFloatParam(
    'conversion_heat', 
    default=26, min=26, max=665,
    distrib=DistributionType.TRIANGLE,
    label='Conversion heat input',
    unit='kWh/kg U in UF6',
    description="Heat required to convert 1 kg of UF6",
    dbname=USER_DB)

In [ ]:
list_parameters() # recap of all parameters declared in the model 

# Transforming and updating parametrized inventories

We need different activities for the different parameters. For example:
    
- An open pit and underground activities for the mining technique mix 
- A diesel machinery and generator
- An electricity grid
- ...

In [ ]:
#copper = [act for act in bd.Database(EI_DB) if
#                     act['name'] == "copper mine operation and beneficiation, sulfide ore" and act['location'] == "CA"][
#    0]
#copper.key


In [ ]:
# Don't know why it finds several activities, so we take the code
copper = findActivity(code='9d722cc3f3dc5522d98392a44fc244b9', loc='CA', db_name=EI_DB)

In [ ]:
# Copy to preserve the non-parametrized inventories 
copper_p = copyActivity(
    USER_DB,
    copper,
    'copper mine operation and beneficiation, sulfide ore, parameterized')

In [ ]:
# list of activities and exchanges, e.g. biosphere and technosphere flows
agb.printAct(copper_p) 

In [ ]:
# Update selected technosphere flow, it is the product name
copper_p.updateExchanges({
    "electricity, high voltage": 15.63*og_cu_world**-0.53
})

In [ ]:
agb.printAct(copper_p) 

# Impact calculation

In [ ]:
# Don't know why we don't have recent methods
iw_methods = [method for method in bd.methods if "world" in " ".join(method).lower()]
iw_methods

In [ ]:
# List of impacts to consider
impacts = agb.findMethods('climate change', mainCat="EF v3.0")
impacts

In [ ]:
# Definition of FU, which can be parametrized
functional_value = 1

In [ ]:
agb.list_databases()

In [ ]:
agb.compute_impacts(
    
    # Root activity of our inventory
    copper_p, 
    
    # list of impacts to consider
    impacts, 
    
    # The impacts will be divided by the functional unit
    functional_unit=functional_value,
    
    # Parameters of the model
)

## OAT sensitivity analysis

In [ ]:
agb.oat_matrix(
    copper_p, 
    impacts, 
    functional_unit=functional_value)

In [ ]:
agb.oat_dashboard(
    copper_p, 
    impacts, 
    functional_unit=functional_value,
    
    # Optionnal layout parameters
    figspace=(0.5,0.5),
    figsize=(15, 15),
    sharex=True)

## GSA

In [ ]:
# Show sobol indices 
agb.incer_stochastic_matrix(
    copper_p, 
    impacts, 
    functional_unit=functional_value)

In [ ]:
agb.incer_stochastic_violin(
    copper_p, impacts,
    functional_unit=functional_value,
    
    # Optionnal layout parameters
    figspace=(0.5,0.5),
    figsize=(15, 15),
    sharex=True, 
    nb_cols=3)

In [ ]:
# Alternatively, graphs can be shown horizontally, together with a box of statistical outcomes
agb.distrib(
    copper_p, impacts,
    functional_unit=functional_value,
    
    # Optionnal layout parameters
    height=7, width=15,
    nb_cols=2,
    percentiles=[5, 95])

In [ ]:
# Full dashboard, including total variation of impacts 
agb.incer_stochastic_dashboard(
    model=copper_p, 
    methods=impacts,
    functional_unit=functional_value)

# Simplified model

In [ ]:
# First, let's look at the full expression defining our model
expr, _ = agb.actToExpression(copper_p)
expr

In [ ]:
simplified = agb.sobol_simplify_model(
    copper_p, # The model
    impacts, # Impacts to consider
    functional_unit=functional_value,
    
    n=10000, # For large model, you may test other value and ensure ST and sum(S1) are close to 1.0 
    fixed_mode = agb.FixedParamMode.MEDIAN, # We replace minor parameters by median by default,
    min_ratio=0.8, # Min ratio of variability to explain
    num_digits=3)

In [ ]:
# Let's look at the expression for first impact again 
# much simpler ! 
simplified[0].expr

In [ ]:
agb.compare_simplified(
    copper_p, 
    impacts, 
    simplified,
    functional_unit=functional_value)